In [1]:
import torch
from torch.nn import functional as F
from config import *
from encoder import Encoder
from modules_rwkv import LLM

In [2]:
def get_loss_lr(log_file):
    with open(log_file) as f:
        lines = f.readlines()

    losses = []
    lrs = []
    for line in lines:
        step = int(line.split()[0][4:-1])
        loss = float(line.split()[1][5:])
        lr = float(line.split()[3][3:])
        if len(losses) > step - 1:
            losses[step - 1] = loss
            lrs[step - 1] = lr
        else:
            losses.append(loss)
            lrs.append(lr)
    return losses, lrs

In [3]:
import matplotlib.pyplot as plt

In [4]:
from dataloader import BinaryDataset, collate_fn
from torch.utils.data import DataLoader

loader = DataLoader(
    BinaryDataset("part-2021278643.json.bin", MAX_LENGTH),
    batch_size=1,
    shuffle=False,
    num_workers=1,
    collate_fn=collate_fn
)
encoder = Encoder.from_path("encoder.json")

In [5]:
llm = LLM(encoder.vocab_size, MODEL_DIM, LORA_DIM, N_BLOCKS, N_HEADS).to("cuda")

In [6]:
length = 256
for x, y, n_tokens in loader:
    x = x.to("cuda")[:, :length, ...]
    y = y.to("cuda")[:, :length, ...]
    break

In [7]:
print(f"{sum(para.numel() for para in llm.parameters())} parameters.")
torch.set_float32_matmul_precision('high')
logits, state = llm(x)
loss = F.cross_entropy(logits.contiguous().view(-1, logits.size(-1)), y.contiguous().view(-1), reduction='mean', ignore_index=0)

60840576 parameters.


In [8]:
out = logits[0].argmax(dim=-1).tolist()[MAX_LENGTH - length:]
print(encoder.decode(x[0].tolist()))
print(encoder.decode(y[0].tolist()))
print(encoder.decode(out))

重庆通报3起违反中央八项规定精神问题：央广网重庆1月31日消息(记者吴新伟)重庆市纪委监察委官微"风正巴渝"今披露,该市近日通报3起违反中央八项规定精神问题。据通报,重庆长寿化工有限责任公司党委委员、董事、副总经理李文成违规发放津补贴。2013年4月至2017年11月,李文成违规以节日津补贴等名义向公司办公室职工发放11万余元,其中本人违规领取3.2万余元。2018年7月,李文成受到党内严重警告处分、扣减2017年全部绩效年薪处理；违纪款项已清退。垫江县原卫生和计划生育委员会党委书记、主任刘卫东违规收受礼金
庆通报3起违反中央八项规定精神问题：央广网重庆1月31日消息(记者吴新伟)重庆市纪委监察委官微"风正巴渝"今披露,该市近日通报3起违反中央八项规定精神问题。据通报,重庆长寿化工有限责任公司党委委员、董事、副总经理李文成违规发放津补贴。2013年4月至2017年11月,李文成违规以节日津补贴等名义向公司办公室职工发放11万余元,其中本人违规领取3.2万余元。2018年7月,李文成受到党内严重警告处分、扣减2017年全部绩效年薪处理；违纪款项已清退。垫江县原卫生和计划生育委员会党委书记、主任刘卫东违规收受礼金问



In [9]:
print(state[0].shape)

torch.Size([1, 96, 768])


In [10]:
loss.backward()